In [2]:
!pip install mesa
import mesa
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.4 MB/s eta 0:00:00


In [12]:
def randomStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = np.random.choice(other_agents)
    return random_card, other_agent

def randomCardStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = np.random.choice(other_agents)
    return random_card, other_agent

In [27]:
def flatten(l):
    return [item for sublist in l for item in sublist]

class Hand:
    def __init__(self, model, agent, other_agents):
        self.model = model
        self.agent = agent
        self.other_agents = other_agents
        self.quartet_groups = [QuartetGroup(quartet_set.group_name, quartet_set.card_names, agent, other_agents)
                              for quartet_set in self.model.quartet_sets]

    def getCards(self, flat = False):
        cards = [quarted_group.getCards() for quarted_group in self.quartet_groups]
        if flat:
            cards = flatten(cards)
        return cards

    def addCard(self, new_card):
        new_card_group_name = new_card.group_name
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == new_card_group_name:
                quartet_group.addCard(new_card)
                return quartet_group.isComplete()

    def missingCards(self, flat = False, exclude_unowned_groups = False):
        missing_cards = [group.missingCards() for group in self.quartet_groups]
        if exclude_unowned_groups:
            new_missing_cards = []
            for missing_cards_group in missing_cards:
                if len(missing_cards_group) != 4:
                    new_missing_cards.append(missing_cards_group)
            missing_cards = new_missing_cards

        if flat:
            missing_cards = flatten(missing_cards)
        return missing_cards

    def getGroup(self, group_name):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == group_name:
                return quartet_group

    def getCard(self, asked_card):
        quartet_group = self.getGroup(asked_card.group_name)
        for card in quartet_group.cards:
            if card.card_name == asked_card.card_name:
                return card

    def giveCard(self, asked_card):
        card = self.getCard(asked_card)
        if card.owned:
            card.owned = False
            return True
        return False

class QuartetGroup:
    def __init__(self, group_name, card_names, agent = -1, other_agents = []):
        self.group_name = group_name
        self.card_names = card_names
        self.agent = agent
        self.other_agents = other_agents
        self.cards = [QuartetCard(group_name, card_name, self) for card_name in card_names]

    def addCard(self, new_card):
        for card in self.cards:
            if new_card == card:
                card.owned = True
                card.potential_owners = [self.agent]

    def missingCards(self):
        current_cards = self.getCards()
        return [x for x in self.cards if x not in current_cards]

    def getCards(self):
        return [x for x in self.cards if x.owned]

    def isComplete(self):
        return np.all([x.owned for x in self.cards])

    def __repr__(self):
        return str([card for card in self.cards])

class QuartetCard:
    def __init__(self, group_name, card_name, quartet_group, owned = False):
        self.group_name = group_name
        self.card_name = card_name
        self.quartet_group = quartet_group
        self.owned = owned
        self.potential_owners = [i for i in quartet_group.other_agents]
        #self.potential_owners.append(-1)

    def __eq__(self, other):
        if not isinstance(other, QuartetCard):
            return NotImplemented

        return self.group_name == other.group_name and self.card_name == other.card_name

    def __repr__(self):
        return self.group_name + "_" + self.card_name

    def __hash__(self):
        return hash((self.group_name, self.card_name))

class QuartetAgent(mesa.Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.other_agents = [i for i in range(self.model.num_agents) if i != self.unique_id]
        self.hand = Hand(model, unique_id, self.other_agents)
        self.score = 0
        self.strategy = randomStrategy


    def step(self):
        print("I am agent", self.unique_id)
        print("I currently have these cards:")
        print(self.hand.getCards())
        print("I currently do not have these cards:")
        print(self.hand.missingCards())

        missing_cards = self.hand.missingCards(flat = True, exclude_unowned_groups = True)
        if len(missing_cards) == 0:
            self.model.skipTurn(self.unique_id)
            return

        asked_card, other_agent = self.strategy(self.hand, self.unique_id, self.other_agents)

        got_card = self.model.askForCard(self.unique_id, other_agent, asked_card)
        if got_card:
            self.addCard(asked_card)
            print("Now, I have these cards:")
            print(self.hand.getCards())

    def addCard(self, new_card):
        has_complete_set = self.hand.addCard(new_card)
        if has_complete_set:
            self.score += 1
            self.model.announceCompleteGroup(self.unique_id, new_card.group_name)


    def giveCard(self, card):
        return self.hand.giveCard(card)



class QuartetModel(mesa.Model):

    def __init__(self, N):
        self.running = True
        self.num_agents = N
        self.schedule = mesa.time.BaseScheduler(self)
        self.quartet_sets = [QuartetGroup('a', ['a_1', 'a_2', 'a_3', 'a_4']),
                             QuartetGroup('b', ['b_1', 'b_2', 'b_3', 'b_4']),
                             QuartetGroup('c', ['c_1', 'c_2', 'c_3', 'c_4'])]
        self.completed_sets = 0
        # Create agents
        for i in range(self.num_agents):
            a = QuartetAgent(i, self)
            self.schedule.add(a)

        all_cards = []
        for quartet_set in self.quartet_sets:
            for quartet_card in quartet_set.cards:
                all_cards.append(quartet_card)
        np.random.shuffle(all_cards)

        cards_per_agent = 4
        for i in range(cards_per_agent):
            for agent in self.schedule.agents:
                current_card = all_cards.pop()
                agent.addCard(current_card)


    def step(self):
        """Advance the model by one step."""
        self.schedule.step()

    def askForCard(self, asking_agent, asked_agent, card):
        print("PA: Agent", asking_agent, "asks agent", asked_agent, "for card", card)
        asked_agent_object = self.schedule.agents[asked_agent]
        has_card = asked_agent_object.giveCard(card)
        if has_card:
            print("PA: agent", asked_agent, "had the card and gave it to", asking_agent)
        else:
            print("PA: agent", asked_agent, "did not have the card.")
        return has_card

    def announceCompleteGroup(self, agent, group_name):
        print("PA: agent", agent, "has completed the set", group_name)
        self.completed_sets += 1
        if len(self.quartet_sets) == self.completed_sets:
            self.finishGame()

    def skipTurn(self, agent):
        print("PA: agent", agent, "cannot ask for any card and skips this turn.")

    def finishGame(self):
        print("----------------------------------------------------------------------------")
        print("The game is finished! Here are the scores:")
        for agent in self.schedule.agents:
            print("Agent", agent.unique_id, "has score", agent.score)
        self.running = False

In [28]:
model = QuartetModel(3)

#for i in range(10):
#    model.step()

model.run_model()

I am agent 0
I currently have these cards:
[[a_a_3, a_a_4], [], [c_c_2, c_c_3]]
I currently do not have these cards:
[[a_a_1, a_a_2], [b_b_1, b_b_2, b_b_3, b_b_4], [c_c_1, c_c_4]]
PA: Agent 0 asks agent 1 for card a_a_1
PA: agent 1 did not have the card.
I am agent 1
I currently have these cards:
[[], [b_b_2, b_b_4], [c_c_1, c_c_4]]
I currently do not have these cards:
[[a_a_1, a_a_2, a_a_3, a_a_4], [b_b_1, b_b_3], [c_c_2, c_c_3]]
PA: Agent 1 asks agent 0 for card c_c_2
PA: agent 0 had the card and gave it to 1
Now, I have these cards:
[[], [b_b_2, b_b_4], [c_c_1, c_c_2, c_c_4]]
I am agent 2
I currently have these cards:
[[a_a_1, a_a_2], [b_b_1, b_b_3], []]
I currently do not have these cards:
[[a_a_3, a_a_4], [b_b_2, b_b_4], [c_c_1, c_c_2, c_c_3, c_c_4]]
PA: Agent 2 asks agent 0 for card b_b_2
PA: agent 0 did not have the card.
I am agent 0
I currently have these cards:
[[a_a_3, a_a_4], [], [c_c_3]]
I currently do not have these cards:
[[a_a_1, a_a_2], [b_b_1, b_b_2, b_b_3, b_b_4], [c